In [1]:
import pandas as pd
import numpy as np
import re
import ast

In [3]:
df = pd.read_csv('ted_main.csv', encoding='iso-8859-1')
df.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [4]:
df['tags'] = df['tags'].apply(ast.literal_eval)

In [52]:
display(df.isnull().sum())

comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    6
tags                  0
title                 0
url                   0
views                 0
tag_views             0
dtype: int64

In [ ]:
#Eliminamos filas cuyo valor de etiquetas o vistas sea nulo
df.

In [30]:
def tags_views(row):
    result = []
    for tag in row['tags']:
        result.append([tag, row['views']])
    return result

df['tag_views'] = df.apply(lambda row: tags_views(row), axis = 1)

for c in df['tag_views'].head(20):
    display(c)

[['children', 47227110],
 ['creativity', 47227110],
 ['culture', 47227110],
 ['dance', 47227110],
 ['education', 47227110],
 ['parenting', 47227110],
 ['teaching', 47227110]]

[['alternative energy', 3200520],
 ['cars', 3200520],
 ['climate change', 3200520],
 ['culture', 3200520],
 ['environment', 3200520],
 ['global issues', 3200520],
 ['science', 3200520],
 ['sustainability', 3200520],
 ['technology', 3200520]]

[['computers', 1636292],
 ['entertainment', 1636292],
 ['interface design', 1636292],
 ['media', 1636292],
 ['music', 1636292],
 ['performance', 1636292],
 ['simplicity', 1636292],
 ['software', 1636292],
 ['technology', 1636292]]

[['MacArthur grant', 1697550],
 ['activism', 1697550],
 ['business', 1697550],
 ['cities', 1697550],
 ['environment', 1697550],
 ['green', 1697550],
 ['inequality', 1697550],
 ['politics', 1697550],
 ['pollution', 1697550]]

[['Africa', 12005869],
 ['Asia', 12005869],
 ['Google', 12005869],
 ['demo', 12005869],
 ['economics', 12005869],
 ['global development', 12005869],
 ['global issues', 12005869],
 ['health', 12005869],
 ['math', 12005869],
 ['statistics', 12005869],
 ['visualizations', 12005869]]

...

[['entertainment', 2702470],
 ['music', 2702470],
 ['performance', 2702470],
 ['violin', 2702470],
 ['wunderkind', 2702470],
 ['youth', 2702470]]

[['creativity', 1628912],
 ['entertainment', 1628912],
 ['music', 1628912],
 ['performance', 1628912],
 ['piano', 1628912],
 ['wunderkind', 1628912]]

[['MacArthur grant', 1415724],
 ['alternative energy', 1415724],
 ['design', 1415724],
 ['engineering', 1415724],
 ['global issues', 1415724],
 ['industrial design', 1415724],
 ['invention', 1415724],
 ['simplicity', 1415724]]

[['DNA', 1074081],
 ['biology', 1074081],
 ['creativity', 1074081],
 ['design', 1074081],
 ['industrial design', 1074081],
 ['invention', 1074081],
 ['nature', 1074081],
 ['product design', 1074081],
 ['science and art', 1074081]]

[['business', 1106561],
 ['collaboration', 1106561],
 ['culture', 1106561],
 ['invention', 1106561],
 ['media', 1106561],
 ['open-source', 1106561],
 ['technology', 1106561],
 ['wikipedia', 1106561]]

In [51]:

first = list(df.loc[0, 'tag_views'])

#tag_data = [row for index, row in df.iterrows() for i in row]
tag_data = []
for index, row in df['tag_views'].iteritems():
    for i in row:
        tag_data.append(i)



tag_views_df = pd.DataFrame(tag_data, columns = ['tags', 'views'])

display(tag_views_df.head(30))
    
    

'''
dftag = pd.DataFrame(first, columns = ['tags', 'views']).set_index()
'''
'''
dftag = pd.DataFrame(first)
dftag = dftag.set_index(dftag.iloc[:, 0])
'''

display(dftag)


#lista = pd.DataFrame(df.loc[0, 'tag_views'])
#display(lista)


'''
df_tags_views = pd.DataFrame.from_dict(list(df['tag_views'])).transpose()
df_tags_views.head()
'''


,tags,views
0,children,47227110
1,creativity,47227110
2,culture,47227110
3,dance,47227110
4,education,47227110
5,parenting,47227110
6,teaching,47227110
7,alternative energy,3200520
8,cars,3200520
9,climate change,3200520


,0,1
0,,
children,children,47227110
creativity,creativity,47227110
culture,culture,47227110
dance,dance,47227110
education,education,47227110
parenting,parenting,47227110
teaching,teaching,47227110


"\ndf_tags_views = pd.DataFrame.from_dict(list(df['tag_views'])).transpose()\ndf_tags_views.head()\n"

In [57]:
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
quandl.ApiConfig.api_key = "FrRQWUBqxyEsq6o3hbUK"

In [58]:
def get_tickers(path):
    companies = pd.read_csv(path)
    ticker_list = list(companies['Ticker'])
    print('Retrieved', str(len(ticker_list)), 'ticker symbols.')
    return ticker_list

def get_prices(ticker):
    print('Retrieving data for', ticker)
    prices = quandl.get('WIKI/' + ticker)['Adj. Close'].reset_index()
    prices['Ticker'] = ticker
    return prices

data = []

ticker_list = get_tickers('companies.csv')

for ticker in ticker_list:
    prices = get_prices(ticker)
    data.append(prices)

Retrieved 30 ticker symbols.
Retrieving data for AAPL
Retrieving data for NFLX
Retrieving data for AMZN
Retrieving data for MSFT
Retrieving data for GOOG
Retrieving data for TSLA
Retrieving data for FB
Retrieving data for AAP
Retrieving data for DIS
Retrieving data for NKE
Retrieving data for UA
Retrieving data for BAC
Retrieving data for CCL
Retrieving data for CI
Retrieving data for AAL
Retrieving data for LUV
Retrieving data for WFC
Retrieving data for WMT
Retrieving data for HD
Retrieving data for ORCL
Retrieving data for IBM
Retrieving data for NCLH
Retrieving data for RCL
Retrieving data for TWX
Retrieving data for FOX
Retrieving data for F
Retrieving data for GCI
Retrieving data for XOM
Retrieving data for WWE
Retrieving data for WM


In [75]:
def get_tickers(path):
    companies = pd.read_csv(path)
    ticker_list = list(companies['Ticker'])
    print('Retrieved', str(len(ticker_list)), 'ticker symbols.')
    return ticker_list

def get_prices(ticker):
    print('Retrieving data for', ticker)
    prices = quandl.get('WIKI/' + ticker)['Adj. Close'].reset_index()
    prices['Ticker'] = ticker
    return prices

def concat_pivot(data, rows, columns, values):

    df = pd.concat(data, sort=True)

    pivot = df.pivot_table(values=values, columns=columns, index=rows)
    return pivot

def compute_returns(df):
    returns = df.pct_change()
    return returns

def return_risk_ratio(df, days=30):
    means = pd.DataFrame(df.tail(days).mean())
    std = pd.DataFrame(df.tail(days).std())
    ratios = pd.concat([means, std], axis=1).reset_index()
    ratios.columns = ['Company', 'Mean', 'Std']
    ratios['Ratio'] = ratios['Mean']/ratios['Std']
    return ratios

def corr_matrix(df, days=30):
    corr_matrix = df.tail(days).corr()
    return corr_matrix

def barchart(df, x, y, length=8, width=14, title=""):
    df = df.sort_values(x, ascending=False)
    plt.figure(figsize=(width,length))
    chart = sns.barplot(data=df, x=x, y=y)
    plt.title(title + "\n", fontsize=16)
    return chart

def correlation_plot(corr, title=""):
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    plt.subplots(figsize=(15, 10))
    cmap = sns.diverging_palette(6, 255, as_cmap=True)
    
    chart = sns.heatmap(corr, mask=mask, cmap=cmap, center=0, linewidths=.5, annot=True, fmt='.2f')
    plt.title(title, fontsize=16)
    return chart

def save_viz(chart, title):
    fig = chart.get_figure()
    fig.savefig(title + '.png')

def acquire():
    data = []

    ticker_list = get_tickers('companies.csv')

    for ticker in ticker_list:
        prices = get_prices(ticker)
        display(type(prices))
        data.append(prices)
    return data

def wrangle(data):
    pivot = concat_pivot(data, 'Date', 'Ticker', 'Adj. Close')
    #display(pivot)
    returns = compute_returns(pivot)
    #display(returns)
    return returns

def analyze(returns, days=30):
    ratios = return_risk_ratio(returns, days=days)
    top10 = ratios.sort_values('Ratio', ascending=False).head(10)
    
    target_list = returns[list(top10['Company'])]
    correlation = corr_matrix(target_list)
    return top10, correlation

def report(top10, correlation):
    bar_plot = barchart(top10, 'Ratio', 'Company', title='Stock Return vs. Risk Ratios - ' + str(day) + ' Days')
    save_viz(bar_plot, 'Return vs. Risk Top 10 - ' + str(day) + ' Days')
    
    corr_plot = correlation_plot(correlation, title='Stock Return Correlation - ' + str(day) + ' Days')
    save_viz(corr_plot, 'Correlation Plot - ' + str(day) + ' Days')

In [72]:
display(data)

[           Date  Adj. Close Ticker
 0    1980-12-12    0.422706   AAPL
 1    1980-12-15    0.400652   AAPL
 2    1980-12-16    0.371246   AAPL
 3    1980-12-17    0.380362   AAPL
 4    1980-12-18    0.391536   AAPL
 5    1980-12-19    0.415355   AAPL
 6    1980-12-22    0.435644   AAPL
 7    1980-12-23    0.454023   AAPL
 8    1980-12-24    0.477841   AAPL
 9    1980-12-26    0.521950   AAPL
 10   1980-12-29    0.529301   AAPL
 11   1980-12-30    0.516363   AAPL
 12   1980-12-31    0.501807   AAPL
 13   1981-01-02    0.507247   AAPL
 14   1981-01-05    0.496220   AAPL
 15   1981-01-06    0.474166   AAPL
 16   1981-01-07    0.454023   AAPL
 17   1981-01-08    0.444760   AAPL
 18   1981-01-09    0.468726   AAPL
 19   1981-01-12    0.464903   AAPL
 20   1981-01-13    0.448436   AAPL
 21   1981-01-14    0.450347   AAPL
 22   1981-01-15    0.459463   AAPL
 23   1981-01-16    0.455787   AAPL
 24   1981-01-19    0.483282   AAPL
 25   1981-01-20    0.468726   AAPL
 26   1981-01-21    0.477841

In [76]:
data = acquire()
returns = wrangle(data)

Retrieved 30 ticker symbols.
Retrieving data for AAPL


pandas.core.frame.DataFrame

Retrieving data for NFLX


pandas.core.frame.DataFrame

Retrieving data for AMZN


...

pandas.core.frame.DataFrame

Retrieving data for AAL


pandas.core.frame.DataFrame

Retrieving data for LUV


KeyboardInterrupt: 

SyntaxError: invalid syntax (<ipython-input-74-6a7f826f7c21>, line 1)